In [1]:
import pandas as pd

In [2]:
path_file = '../input/onegin/evgenyi_onegin.txt'

In [3]:
text = open(path_file, 'rb').read().decode(encoding='utf-8')

In [4]:
def build_text_files(data_path, dest_path):
    f = open(dest_path, 'w')
    data = ''
    for texts in data_path:
        summary = str(texts)
        data += summary 
    f.write(data)

In [5]:
build_text_files(text,'train_dataset.txt')

In [6]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")

In [7]:
train_path = 'train_dataset.txt'

In [8]:
from transformers import TextDataset, DataCollatorForLanguageModeling

def load_dataset(data, tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset, data_collator

train_dataset, data_collator = load_dataset(train_path, tokenizer)

In [9]:
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [10]:
from transformers import Trainer, TrainingArguments, AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2").to(device)

In [11]:
training_args = TrainingArguments(
    output_dir="./gpt2-chief", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=5, # number of training epochs
    per_device_train_batch_size=4, # batch size for training
    per_device_eval_batch_size=4,  # batch size for evaluation
    eval_steps = 400, # Number of update steps between two evaluations.
    save_steps=800, # after # steps model is saved
    warmup_steps=200,# number of warmup steps for learning rate scheduler
    )

In [12]:
len(train_dataset)

In [13]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
)

In [14]:
trainer.train()

In [15]:
trainer.save_model('model_compose')

In [16]:
tokenizer.save_pretrained('tokenizer_compose')

In [17]:
tokenizer.save_pretrained('tokenizer_compose')
model.save_pretrained('model_compose')

In [18]:
tokenizer = AutoTokenizer.from_pretrained("tokenizer_compose")
model = AutoModelForCausalLM.from_pretrained("model_compose")

In [ ]:
model.save('model_compose.h5')


In [ ]:
prefix = "однажды"

In [ ]:
tokens = tokenizer(prefix, return_tensors='pt')

In [ ]:
size = tokens['input_ids'].shape[1]
output = model.generate(
    **tokens, 
    #end_token=end_token_id,
    do_sample=False,
    
    max_length=size+200, 
    repetition_penalty=5., 
    temperature=0.8,
    num_beams=10,
)

decoded = tokenizer.decode(output[0])
result = decoded[len(prefix):]
print(prefix + result)